In [72]:
!pip install evaluate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.4 MB/s eta 0:00:00


In [57]:
!pip install transformers datasets torch
!pip install fsspec==2025.3.2



  Using cached fsspec-2025.3.2-py3-none-any.whl.metadata (11 kB)
Using cached fsspec-2025.3.2-py3-none-any.whl (194 kB)
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.12.0
    Uninstalling fsspec-2024.12.0:
      Successfully uninstalled fsspec-2024.12.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 1.42.0 requires gcsfs>=2023.3.0, which is not installed.
datasets 3.5.0 requires fsspec[http]<=2024.12.0,>=2023.1.0, but you have fsspec 2025.3.2 which is incompatible.


In [58]:
!pip uninstall fsspec -y


Found existing installation: fsspec 2025.3.2
Uninstalling fsspec-2025.3.2:
  Successfully uninstalled fsspec-2025.3.2


In [59]:
!pip install fsspec==2024.12.0


  Using cached fsspec-2024.12.0-py3-none-any.whl.metadata (11 kB)
Using cached fsspec-2024.12.0-py3-none-any.whl (183 kB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 1.42.0 requires gcsfs>=2023.3.0, which is not installed.


In [60]:
!pip uninstall gcsfs -y


In [61]:
!pip install fsspec==2024.12.0
!pip install --upgrade datasets


In [62]:
!pip install sentencepiece accelerate huggingface_hub scikit-learn

In [63]:
from datasets import load_dataset

# Cargar el dataset LIAR con la opción de confiar en el código remoto
dataset = load_dataset("liar", trust_remote_code=True)



In [64]:
num_classes = len(set(dataset["train"]["label"]))
print(f"Número de clases: {num_classes}")


Número de clases: 6


In [65]:
# Cargar el tokenizador de TinyBERT
tokenizer = AutoTokenizer.from_pretrained("huawei-noah/TinyBERT_General_4L_312D")

# Tokenizar el dataset
def tokenize_function(examples):
    # Access the correct column name for the input text.
    # The 'liar' dataset uses 'statement' for the text data.
    return tokenizer(examples["statement"], padding="max_length", truncation=True, max_length=35)

# Apply tokenization to the whole dataset
tokenized_datasets = dataset.map(tokenize_function, batched=True)



Map:   0%|          | 0/1283 [00:00<?, ? examples/s]

In [66]:
# Ver los primeros ejemplos tokenizados
print(tokenized_datasets["train"][0])  # O cualquier otra división del dataset


{'id': '2635.json', 'label': 0, 'statement': 'Says the Annies List political group supports third-trimester abortions on demand.', 'subject': 'abortion', 'speaker': 'dwayne-bohac', 'job_title': 'State representative', 'state_info': 'Texas', 'party_affiliation': 'republican', 'barely_true_counts': 0.0, 'false_counts': 1.0, 'half_true_counts': 0.0, 'mostly_true_counts': 0.0, 'pants_on_fire_counts': 0.0, 'context': 'a mailer', 'input_ids': [101, 2758, 1996, 8194, 2015, 2862, 2576, 2177, 6753, 2353, 1011, 12241, 20367, 11324, 2015, 2006, 5157, 1012, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}


In [67]:
# Dividir el dataset (si es necesario)
train_dataset = tokenized_datasets["train"]
test_dataset = tokenized_datasets["test"]


In [68]:
from transformers import AutoModelForSequenceClassification

num_classes = 6  # Asegurar que el modelo tenga 6 clases

model = AutoModelForSequenceClassification.from_pretrained(
    "huawei-noah/TinyBERT_General_4L_312D",
    num_labels=num_classes  # Asegura que el modelo tenga la cantidad correcta de etiquetas
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at huawei-noah/TinyBERT_General_4L_312D and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [69]:
import os
os.environ["WANDB_DISABLED"] = "true"


In [73]:
from transformers import Trainer, TrainingArguments
import numpy as np
import evaluate

# Definir los argumentos de entrenamiento con la estrategia de guardado ajustada
training_args = TrainingArguments(
    output_dir="./results",  # Directorio de salida para los resultados
    num_train_epochs=3,      # Número de épocas
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs",
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    report_to="none"  # No reportes a W&B
)


# Cargar la métrica de exactitud
metric = evaluate.load("accuracy")


# Definir una función compute_metrics
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)  # Obtener la clase predicha
    return metric.compute(predictions=predictions, references=labels)


# Definir el entrenador
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics  # Agregar métricas
)


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


<ipython-input-73-df650e42389a>:31: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [74]:
# Entrenar el modelo
trainer.train()


Epoch,Training Loss,Validation Loss,Accuracy
1,1.695900,1.736858,0.239875
2,1.639100,1.700739,0.257788
3,1.590200,1.718379,0.252336


TrainOutput(global_step=3852, training_loss=1.6853896972422535, metrics={'train_runtime': 1802.6602, 'train_samples_per_second': 17.09, 'train_steps_per_second': 2.137, 'total_flos': 30205269666180.0, 'train_loss': 1.6853896972422535, 'epoch': 3.0})